In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import glob
import json
import matplotlib.pyplot as plt
from tqdm import tqdm
from langdetect import detect
from langdetect import DetectorFactory
from pprint import pprint
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
import en_core_sci_lg
import os
import gensim
import gensim.corpora as corpora

In [121]:
#authors

with open("authors_abstracts.json", 'r', encoding='utf-8') as author:
    authors = json.load(author)

In [122]:
len(authors)

12482

In [116]:
#papers

with open("papers1.json", 'r', encoding='utf-8') as papers:
    papers = json.load(papers)

In [123]:
len(papers)

12482

In [127]:
author_and_abstracts = {}
with open("authors_abstracts.json", 'r', encoding='utf-8') as author, open("papers1.json", 'r', encoding='utf-8') as papers:
    authors = json.load(author)
    papers = json.load(papers)
    for authID in authors:
        if 'authorId' in authID:
            key = str(authID['authorId'])
            author_and_abstracts.setdefault(key, [])
            #print("finding ID:", authID['authorId'])
            for j in papers:
                if j:
                    try:        
                        for k in j["authors"]:
                            if k["authorId"] == key:
                                author_and_abstracts[key].append(j["abstract"])
                            else:
                                continue
                    except:   
                        #print the error message from sys
                        print("error:", sys.exc_info()[0])
                else:
                    continue
        else:
            continue

In [128]:
author_and_abstracts

{'2108425749': [],
 '39260960': [],
 '49873115': [],
 '4274593': [],
 '1400741849': [],
 '4994265': [],
 '2114068408': [],
 '83504588': [],
 '143627454': ['Protecting Health Care Workers (HCWs) during routine care of suspected or confirmed COVID-19 patients is of paramount importance to halt the SARS-CoV-2 (Severe Acute Respiratory Syndrome-Coronavirus-2) pandemic. The WHO, ECDC and CDC have issued conflicting guidelines on the use of respiratory filters (N95) by HCWs. We searched PubMed, Embase and The Cochrane Library from the inception to March 21, 2020 to identify randomized controlled trials (RCTs) comparing N95 respirators versus surgical masks for prevention of COVID-19 or any other respiratory infection among HCWs. The grading of recommendations, assessment, development, and evaluation (GRADE) was used to evaluate the quality of evidence. Four RCTs involving 8736 HCWs were included. We did not find any trial specifically on prevention of COVID-19. However, wearing N95 respirato

In [129]:
len(author_and_abstracts)

12460

In [133]:
print(len(author_and_abstracts['46996499']))

3


In [135]:
auth_abs = pd.DataFrame.from_dict(author_and_abstracts, orient='index')

In [157]:
auth_abs

,0,1,2,3,4,5,6,7
2108425749,None,None,None,None,None,None,None,None
39260960,None,None,None,None,None,None,None,None
49873115,None,None,None,None,None,None,None,None
4274593,None,None,None,None,None,None,None,None
1400741849,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...
2068126911,None,None,None,None,None,None,None,None
3911061,None,None,None,None,None,None,None,None
152113001,None,None,None,None,None,None,None,None
46195081,None,None,None,None,None,None,None,None


In [161]:
auth_abs

,0,1,2,3,4,5,6,7
2108425749,None,None,None,None,None,None,None,None
39260960,None,None,None,None,None,None,None,None
49873115,None,None,None,None,None,None,None,None
4274593,None,None,None,None,None,None,None,None
1400741849,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...
2068126911,None,None,None,None,None,None,None,None
3911061,None,None,None,None,None,None,None,None
152113001,None,None,None,None,None,None,None,None
46195081,None,None,None,None,None,None,None,None


In [162]:
new_df = auth_abs.dropna(how='all')

In [163]:
new_df

,0,1,2,3,4,5,6,7
143627454,Protecting Health Care Workers (HCWs) during r...,None,None,None,None,None,None,None
4047287,\n There is increasing evidence that ACE2 gene...,None,None,None,None,None,None,None
1398339345,"The liver-derived hormone hepcidin, a member o...",PURPOSE OF REVIEW\nThe COVID-19 infection resu...,None,None,None,None,None,None
50177639,"Background Compared to adults, there are relat...",BackgroundViral infection involves a large num...,None,None,None,None,None,None
1557669371,\n Abstract\n \n To better understand the gene...,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...
98852929,SUMMARY A study was conducted in the USA to de...,None,None,None,None,None,None,None
144252495,Timeliness of global outbreak discovery and pu...,None,None,None,None,None,None,None
4683017,The outbreak of the novel 2019 SARS2-Coronavir...,None,None,None,None,None,None,None
143966135,Microfluidic organ-on-a-chip (Organ Chip) cell...,Lymphoid follicles (LFs) are responsible for g...,None,None,None,None,None,None


In [ ]:
new_df = new_df['0'] + new_df['1'] + new_df['2'] + new_df['3'] + new_df['4'] + new_df['5'] + new_df['6'] + new_df['7']

In [70]:
data[0]

{'aliases': ['Jiang-lin Fan', 'Jianglin Fan'],
 'authorId': '13382020',
 'dblpId': None,
 'influentialCitationCount': 4,
 'name': 'Jiang-lin Fan',
 'papers': [{'paperId': '939b3e42c0033c7191df2e9a51be266e08ff8615',
   'title': 'Perfluorooctane sulfonate impairs the cardiac development of a marine medaka (Oryzias melastigma).',
   'url': 'https://www.semanticscholar.org/paper/939b3e42c0033c7191df2e9a51be266e08ff8615',
   'year': 2011},
  {'paperId': '4f09563974930a99ebb1f1416244c03c6d90b10b',
   'title': 'Immunogenicity of SARS inactivated vaccine in BALB/c mice',
   'url': 'https://www.semanticscholar.org/paper/4f09563974930a99ebb1f1416244c03c6d90b10b',
   'year': 2004},
  {'paperId': 'f17b63a65f6be7953fa4f047365db8c8231fd482',
   'title': 'Genetically Modified Rabbits for Cardiovascular Research',
   'url': 'https://www.semanticscholar.org/paper/f17b63a65f6be7953fa4f047365db8c8231fd482',
   'year': 2021},
  {'paperId': '1b9d892943f48433bc0e1fa08ce2979e566dcff4',
   'title': 'Increased

In [72]:
data[0]['papers'][0]['paperId']

'939b3e42c0033c7191df2e9a51be266e08ff8615'

We want to extract the author ID, and their respective paper IDs

Make a DF that contains two columns: authorId and paperId, where author has IDs, and paperIDs are a list of IDs

In [84]:
auth_paper = {}

for i in range(0,len(data)):
    keys = data[i]['authorId']
    for j in range(0,len(data[i]['papers'])):
        values = data[i]['papers'][j]['paperId']
        
    auth_paper[keys] = values 
        
        
#     for values in range(0,len(data[i]['papers'][j]['paperId'])):
           

In [85]:
auth_paper

{'13382020': '1ee3d3cf45636caae05350c1295e5bfb4b5831af',
 '2406109': 'da57660879e5bbb55bf9bca9292816d40d93e842',
 '143938360': '146ea11000307c5ff6636295bdb9bc5ac587f275',
 '2215115': 'd96fc4bfac400a3a373f1113e5571d61c127f4ce',
 '2235791': '45a610297e9409cf6213ef67011dce90f0b4a01d'}

In [69]:
print(author_id)

['13382020', '2406109', '143938360', '2215115', '2235791']


In [67]:
print(paper_id)

[['939b3e42c0033c7191df2e9a51be266e08ff8615'], ['4f09563974930a99ebb1f1416244c03c6d90b10b'], ['f17b63a65f6be7953fa4f047365db8c8231fd482'], ['1b9d892943f48433bc0e1fa08ce2979e566dcff4'], ['458be46a7d7bfcbb8efb5b8dc05a3675ea31399e'], ['8d4e80b490bc54f1c37c07c0efd9460c0a4625b3'], ['1ee3d3cf45636caae05350c1295e5bfb4b5831af'], ['1bcfae3e4f8d6da3908688821d6d717788beb7cc'], ['0597d604ca9b5b9f9c3d16b858cc511bfb072d72'], ['531dfbe11bdeefaeb59abefcc9da9c79bc92e4bf'], ['2a97eb0801462637f4fa9d9795086bcea9733c65'], ['e05ee028cd8d54e58a38651baafb5350ba505613'], ['e5d47cced0d779dd0a6980c4c13b628e4c7592e2'], ['66e6a333f24d41d4e0313e361b875bda32aed758'], ['1261039cf5fdbacaee9514a23f6a2efbf1069e95'], ['a80a7f393808f92703ebfad58ab31bd982f23c09'], ['f1265272a6a7012d665785b6ff81a6ee9a157723'], ['0d16fbe80e807df57364c88f49e652e2999a34c1'], ['acaf48db824fb402c0acfd02a062dbe7fd9c911f'], ['5573239b043f58ffc213a2c5f1e9eb494b1d847e'], ['ca516169f073d460003af117e7afdb50c00a53e1'], ['da57660879e5bbb55bf9bca9292816d

In [20]:
df = pd.json_normalize(data)

In [21]:
df.columns = df.columns.map(lambda x: x.split(".")[-1])

In [31]:
df['authorId']

0     13382020
1      2406109
2    143938360
3      2215115
4      2235791
Name: authorId, dtype: object